**Leitos de UTI por município, sendo ou não do SUS , dados do DataSUS**

Libraries

In [0]:
import pandas as pd
import numpy as np

###Leitura de CSV's

Leitos por município

In [3]:
leitos_uti_sus_nao_sus = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/leitos_uti_sus_nao-sus_municipio.csv", sep = ";", encoding = "iso-8859-1")
leitos_uti_sus_nao_sus.columns = ["city","quantidade_existente","quantidade_sus","quantidade_nao-sus"]
leitos_uti_sus_nao_sus.head(2)

,city,quantidade_existente,quantidade_sus,quantidade_nao-sus
0,110002 Ariquemes,19,7,12
1,110004 Cacoal,28,18,10


Dados de Lagitude e Longitude dos municípios 

In [4]:
city_lat_long =  pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/lat_long_municipios.csv")
city_lat_long.drop(columns = ["capital","Unnamed: 0"], inplace = True)
city_lat_long.rename(columns = {"nome":"city"},inplace = True)
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31


Base do IBGE, contendo código do município, nome do estado, do município, e o código do estado

In [5]:
data_ibge = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/dados_ibge.csv")
data_ibge.drop(columns = ["Mesorregião Geográfica","Nome_Mesorregião","Microrregião Geográfica","Nome_Microrregião","Município","Unnamed: 0"], inplace = True)
data_ibge.rename(columns = {"Nome_Município":"city","Nome_UF":"state","UF":"codigo_uf","Código Município Completo":"codigo_ibge"},inplace = True)
data_ibge.head(2)

,codigo_uf,state,codigo_ibge,city
0,11,Rondônia,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1100379,Alto Alegre dos Parecis


###Manipulação de dataframes

Juntando a tabela da localização geográfica com a dos dados do ibge

In [6]:
city_lat_long = city_lat_long.merge(data_ibge, on = ["city","codigo_ibge","codigo_uf"],sort = "city")
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Usando Split para retirar o código do município

In [7]:
new = leitos_uti_sus_nao_sus["city"].str.split(" ", n = 1, expand = True)
leitos_uti_sus_nao_sus["codigo_ibge"] = new[0] #Dev sumiu com o ultimo digito de todos os codigos
leitos_uti_sus_nao_sus.drop(columns=["city"], inplace = True)
leitos_uti_sus_nao_sus.head(2)

,quantidade_existente,quantidade_sus,quantidade_nao-sus,codigo_ibge
0,19,7,12,110002
1,28,18,10,110004


Retirando o último digito para manter a igualdade, já que os dados da datasus vieram faltando o útltimo dígito

In [8]:
city_lat_long["codigo_ibge"] = city_lat_long["codigo_ibge"].map(lambda x: str(x)[:-1])
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,520005,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,310010,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Merge com a localização 

In [9]:
leitos_uti_sus_nao_sus = leitos_uti_sus_nao_sus.merge(city_lat_long, on = ["codigo_ibge"], sort = "city", how = "outer")
leitos_uti_sus_nao_sus.dropna(subset=["state"], inplace = True)
leitos_uti_sus_nao_sus.head(2)

,quantidade_existente,quantidade_sus,quantidade_nao-sus,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,NaN,NaN,NaN,110001,Alta Floresta D'Oeste,-11.92830,-61.9953,11.0,Rondônia
1,19.0,7,12,110002,Ariquemes,-9.90571,-63.0325,11.0,Rondônia


Retirando valores nulos

In [12]:
leitos_uti_sus_nao_sus.replace("-", 0, inplace = True)
leitos_uti_sus_nao_sus.replace(np.nan, 0, inplace = True)
leitos_uti_sus_nao_sus.head(2)

,quantidade_existente,quantidade_sus,quantidade_nao-sus,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,0.0,0,0,110001,Alta Floresta D'Oeste,-11.92830,-61.9953,11.0,Rondônia
1,19.0,7,12,110002,Ariquemes,-9.90571,-63.0325,11.0,Rondônia


###Download CSV

In [0]:
leitos_uti_sus_nao_sus.to_csv("leitos_uti_sus_nao_sus.csv")